# CS295B F19: Homework 4
## Local Sensitivity, Propose-test-release, and Sample and Aggregate

## Instructions

Before you start, download the example dataset and ensure that all cells in this notebook execute without error. If you have trouble getting the notebook to run, please post a question on Piazza.

To ensure that the notebook runs, I've defined a function `your_code_here()` that simply returns the number `1`. Whenever you see a call to this function, you should replace it with code you have written. Please make sure all cells of your notebook run without error before submitting the assignment. If you have not completed all the questions, leave calls to `your_code_here()` in place or insert dummy values so that the cell does not throw an error when it runs.

To help you arrive at the correct solution, I have left the value computed by my solution in the uploaded version of this notebook. You can refer to these example results by viewing the notebook on Github. If you re-run the cell after downloading the notebook, the results will disappear (because the notebook no longer contains the code that generated them). Your solutions should produce results similar to the ones in the uploaded notebook.

When answering non-code questions, feel free to use a comment, or put the cell in Markdown mode and use Markdown.

The assignment is due by 5:00pm on Friday, October 18. When you have finished your assignment, submit it via Gradescope under the assignment "Homework 4." For questions on grading and submitting assignments, refer to the course webpage or email the instructor.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import pandas as pd
import numpy as np

# Some useful utilities

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

def z_clip(xs, b):
    return [min(x, b) for x in xs]

def clip(xs, upper, lower):
    return [max(min(x, upper), lower) for x in xs]

def your_code_here():
    return 1

def test(msg, value, expected):
    if value == expected:
        print(f"{msg}: {value}, as expected")
    else:
        print(f"{msg}: OH NO! Got {value}, but expected {expected}.")

In [2]:
adult_data = pd.read_csv("adult_with_pii.csv", parse_dates=['DOB'])

## Question 1 (5 points)
Consider the following dataframes and the query `count_join`.

In [4]:
df1 = pd.DataFrame(data={'k': [1, 1, 1], 'v1': ['a', 'b', 'c']})
df2 = pd.DataFrame(data={'k': [1, 1, 2], 'v2': ['C', 'D', 'E']})
display(df1)
display(df2)

def count_join(df1, df2):
    return pd.merge(df1, df2, left_on='k', right_on='k').shape[0]

count_join(df1, df2)

,k,v1
0,1,a
1,1,b
2,1,c


,k,v2
0,1,C
1,1,D
2,2,E


6

 What is the local sensitivity of `count_join(df1, df2)`, and why?
 
 *Hint*: argue in terms of the "worst possible change" to the true database (`df1` and `df2`).

## Question 2 (20 points)

Consider the following definitions, which provide functions for computing the local sensitivity for a counting query over a join at an arbitrary distance $k$.

In [ ]:
def max_freq(df):
    return int(df['k'].value_counts().to_frame().max())

def ls_count_join_k(df1, df2, k):
    return max(k + max_freq(df1), k + max_freq(df2))

Write code that uses the propose-test-release framework to release a differentially private answer to the query `count_join`. Your solution should return `False` when the test phase of propose-test-release fails.

*Hint*: use the definition of `ls_count_join_k` to compute the local sensitivity at distance $k$.

In [9]:
def ptr_count_join(df1, df2, b, epsilon, delta):
    return your_code_here()

# Note: your answers will differ due to randomness
print(f"b = 1: {ptr_count_join(df1, df2, 1, 1, 1e-5)}")
print(f"b = 20: {ptr_count_join(df1, df2, 20, 1, 1e-5)}")
print(f"b = 50: {ptr_count_join(df1, df2, 50, 1, 1e-5)}")

b = 1: False
b = 20: 6.60113346045531
b = 50: -43.20378979756899


## Question 3 (20 points)

Consider the query `capgain_mean`, which calculates the mean of the `Capital Gain` column of the input dataframe.

In [11]:
def capgain_mean(df):
    return np.mean(df['Capital Gain'])

capgain_mean(adult_data)

1077.6488437087312

Write code that implements sample-and-aggregate for the `capgain_mean` query.

*Hint*: use `np.array_split` to split `df` into chunks.

*Hint*: use `clip` (defined above) to clip results.

In [12]:
def saa_capgain_mean(df, k, upper, lower, epsilon):
    return your_code_here()

print(f"k = 20: {saa_capgain_mean(adult_data, 20, 10000, 0, 1)}")
print(f"k = 200: {saa_capgain_mean(adult_data, 200, 10000, 0, 1)}")
print(f"k = 2000: {saa_capgain_mean(adult_data, 2000, 10000, 0, 1)}")

k = 20: 1086.8939395457935
k = 200: 1085.6258111645795
k = 2000: 1068.951971027325


## Question 4 (10 points)

Run the tests in Question 3 several times and observe the accuracy for each value of $k$. What value of $k$ seems to work the best? Describe the tradeoff in setting a value for $k$.

## Question 5 (5 points)

Do you think sample and aggregate would work well for join queries like `count_join`? Why, or why not?